In [1]:
import os 
import getopt
import sys

import random
import re

import numpy as np
import h5py
import pickle
import random
import copy
import pandas as pd
import math 

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Masking, Lambda, concatenate, Bidirectional, Dense, Dropout, Flatten, Conv1D,BatchNormalization,  MaxPooling1D, Bidirectional, GRU, TimeDistributed
import tensorflow as tf
from tensorflow import keras



np.random.seed(1337) # for reproducibility
vocab = ["A", "G", "C", "T"]
indices = tf.range(len(vocab), dtype = tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab,indices)
table = tf.lookup.StaticVocabularyTable(table_init, 1)
record_defaults = [
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),  
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),  
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
]

# Nadav dataset

def data_reader(file, batch_size=100, n_parse_threads=4):
    """Method for reading the data in an optimized way, can be used inside model.fit()
    
    Args:
        file (_type_): path to csv file
        batch_size (int, optional): _description_. Defaults to 100.
        n_parse_threads (int, optional): _description_. Defaults to 4.

    Returns:
        dataset.batch: batch dataset object 
    """
    dataset = tf.data.TextLineDataset(file).skip(1)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)

def preprocess(record):
    """Preprocessing method of a dataset object, one-hot-encodes the data

    Args:
        record (_type_): _description_

    Returns:
        X (2D np.array): one-hot-encoded input sequence
        Y (1D np.array): MPRA measurements for each cell state
    """
    # Get fields from the data
    fields = tf.io.decode_csv(record, record_defaults=record_defaults)
    
    # One-hot-encode data
    chars = tf.strings.bytes_split(fields[0])
    chars_indeces = table.lookup(chars)
    X = tf.one_hot(chars_indeces, depth = len(vocab))
    
    # Combine y for each cell type into one vector 
    Y = tf.stack(fields[2:])
    
    # Replace missing values with -1
    Y= tf.where(tf.equal(Y,  "nan"), ["-1"], Y)
    Y= tf.where(tf.equal(Y,  "NA"), ["-1"], Y)
    Y = tf.strings.to_number(Y, tf.float32)
    return X,Y

# Get first item of the dataset to get the shape of the input data
for element in data_reader("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/upper_LibH_wide_pivot_all_states.csv"):
    input_shape = element[0].shape
    output_shape = element[1].shape
    
print(output_shape)
print(input_shape)

2023-12-04 13:06:42.603149: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-04 13:06:42.636288: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 13:06:42.636330: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 13:06:42.636360: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 13:06:42.643209: I tensorflow/core/platform/cpu_feature_g

(67, 7)
(67, 246, 4)


In [2]:
# CROSS VALIDATION (10 fold)
from sklearn.model_selection import train_test_split, KFold

whole_data = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/upper_LibH_wide_pivot_all_states.csv"
out_data = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/"
# Split the data in two partitions
whole_data = pd.read_csv(whole_data)
k = 10
kf = KFold(n_splits = k, shuffle = True, random_state = 2008)

o=1
# For each fold we split again to get the third partition
for i in kf.split(whole_data):
    
    # Extract data from fold
    train = whole_data.iloc[i[0]]
    test =  whole_data.iloc[i[1]]
    
    # Split into train, validation and test
    valid, validation = train_test_split(train, test_size=0.111, random_state=42)
    
    # Randomize motifs
    #train = randomize_motifs_and_augment(train, num_augmentations=100)
    #train["rnd_seq"] = train['rnd_seq'].str.lower() 
    #test = randomize_motifs_and_augment(test, num_augmentations=100)
    #test["rnd_seq"] = test['rnd_seq'].str.lower() 
    #validation = randomize_motifs_and_augment(validation, num_augmentations=100)
    #validation["rnd_seq"] = validation['rnd_seq'].str.lower() 
    
    train.astype(str).to_csv(out_data+"all_states_CV"+str(o)+"_LibH_wide_pivot_state3_train.csv", index=False)
    test.astype(str).to_csv(out_data+"all_states_CV"+str(o)+"_LibH_wide_pivot_state3_test.csv", index=False)
    validation.astype(str).to_csv(out_data+"all_states_CV"+str(o)+"_LibH_wide_pivot_state3_validation.csv", index=False)
    o+=1
    

In [3]:
train

,Seq,CRS,State_1M,State_2D,State_3E,State_4M,State_5M,State_6N,State_7M
0,AGGACCGGATCAACTAAAAAAATGGTAACTTGGTCCTAGGCGATTA...,LibH.Seq13360,0.029339,0.075195,0.009291,0.020216,0.068678,0.152887,-0.008230
1,AGGACCGGATCAACTAAAAAACCCAAGAGCAGCAGCTCGTAAGATA...,LibH.Seq16871,0.045097,0.005560,0.017385,0.043642,0.011030,0.148327,-0.024262
2,AGGACCGGATCAACTAAAAAACGCCGGTGCAACTGGATGTGCTATC...,LibH.Seq829,-0.193690,-0.177044,-0.185218,-0.114833,NaN,NaN,-0.281635
3,AGGACCGGATCAACTAAAAAATATTCCAATATCCCGAAGGAATGTT...,LibH.Seq17946,-0.022712,0.150933,0.167140,0.016367,0.170019,0.062101,0.512236
4,AGGACCGGATCAACTAAAAAATGGAAAACCATATTAGGTCGCGTTG...,LibH.Seq10013,-0.125909,-0.132108,-0.121524,-0.108408,-0.181684,-0.078825,-0.106308
...,...,...,...,...,...,...,...,...,...
20461,AGGACCGGATCAACTTTTTTGGATGGATTAAGTTAAGGACCCGGCC...,LibH.Seq18634,0.016718,0.066757,-0.045963,0.056641,0.006334,-0.048728,-0.020860
20462,AGGACCGGATCAACTTTTTTGGGAGCTCCCGTAACGGCATTTCGGG...,LibH.Seq11461,0.397537,0.077166,0.188109,0.077516,0.098549,NaN,0.114883
20463,AGGACCGGATCAACTTTTTTTACCAAGGTCGGTCCTTTGTACGACA...,LibH.Seq9475,-0.019392,-0.016249,0.052844,0.145790,0.118632,0.068189,-0.033873
20465,AGGACCGGATCAACTTTTTTTCTACTATGAGGCTCCAGAATTGGGA...,LibH.Seq852,0.005635,-0.043291,-0.058870,-0.043434,-0.145355,-0.021086,-0.016615


In [4]:
# Here we initialize the df where each fold test prediction will be appended to
# the list containing the correlations of each fold is also initialized
corr_list = []

# We define a custom normalization layer to then compile on the model
class CustomNormalization(Layer):
    """Custom normalization layer that normalizes the output of the neural network"""
    def __init__(self, **kwargs):
        super(CustomNormalization, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Add trainable variables for mean and standard deviation
        self.mean = self.add_weight("mean", shape=(1,), initializer="zeros", trainable=True)
        self.stddev = self.add_weight("stddev", shape=(1,), initializer="ones", trainable=True)
        super(CustomNormalization, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        # Normalize the inputs using the learned mean and standard deviation
        return (inputs - self.mean) / (self.stddev + 1e-8)

# We define the method to compute the pearson correlation between prediction and ground truth in the multi_head case
def pearson_correlation_multi_head(predictions, ground_truth, mask_value=-1):
    """Computes Pearson Correlation between predictions and ground truth for each column
    Args:
        predictions (np.array): 2D array of prediction values (N, 7)
        ground_truth (np.array): 2D array of ground truth values (N, 7)
        mask_value (float): Value in ground truth to be ignored in correlation computation

    Returns:
        correlations (np.array): 1D array of Pearson correlations for each column
    """
    # Ensure predictions and ground_truth have the same shape
    assert predictions.shape == ground_truth.shape, "Input shapes do not match"

    n_columns = predictions.shape[1]
    correlations = np.zeros(n_columns)

    for col in range(n_columns):
        x = predictions[:, col]
        y = ground_truth[:, col]

        # Exclude values in ground truth equal to mask_value
        valid_indices = (y != mask_value)
        x = x[valid_indices]
        y = y[valid_indices]

        if len(x) == 0 or len(y) == 0:
            # If no valid values, set correlation to NaN
            correlations[col] = np.nan
        else:
            # Calculate mean of x and y
            mean_x = np.mean(x)
            mean_y = np.mean(y)

            # Calculate the numerator and denominators of the correlation coefficient
            numerator = np.sum((x - mean_x) * (y - mean_y))
            denominator_x = np.sqrt(np.sum((x - mean_x) ** 2))
            denominator_y = np.sqrt(np.sum((y - mean_y) ** 2))

            # Calculate the correlation coefficient
            correlation = numerator / (denominator_x * denominator_y)
            correlations[col] = correlation

    return correlations



import matplotlib.pyplot as plt
%matplotlib inline
# Define plotting function of loss
def create_plots(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.clf()
    

In [5]:
# Define a custom loss function
class MaskedMSE(tf.keras.losses.Loss):
    """Computes the MSE loss and prevents missing values backpropagation (previously replaced by -1.0)

    Args:
        tf (_type_): _description_
    """
    def __init__(self, mask_value=-1, **kwargs):
        super(MaskedMSE, self).__init__(**kwargs)
        self.mask_value = mask_value

    def call(self, y_true, y_pred):
        # Create a mask for valid elements (not equal to the specified mask_value)
        mask = tf.math.not_equal(y_true, self.mask_value)

        # Compute MSE loss only for valid elements
        loss = tf.where(mask, tf.square(y_true - y_pred), 0.0)

        # Calculate the mean loss
        mean_loss = tf.reduce_sum(loss) / tf.reduce_sum(tf.cast(mask, dtype=tf.float32))

        return mean_loss

In [6]:
from tensorflow.keras.optimizers import Adam

corr_coefficients = {}

df_test_10folds  = pd.DataFrame(columns=['State_3E',
                                         "seq",
                                         "avg_prediction_State_1M",
                                         "avg_prediction_State_2D",
                                         "avg_prediction_State_3E",
                                         "avg_prediction_State_4M",
                                         "avg_prediction_State_5M",
                                         "avg_prediction_State_6N",
                                         "avg_prediction_State_7M"])

# We iterate through each of the train folds to train, test and validate the model

for i in range(1,11):
    #Define inputs
    input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_CV"+str(i)+"_LibH_wide_pivot_state3_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_CV"+str(i)+"_LibH_wide_pivot_state3_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_CV"+str(i)+"_LibH_wide_pivot_state3_test.csv"
    
    # Read test data to then predict
    df_test = pd.read_csv(input_path_test)
    df_test["fold"] = str(i)
    
    predictions_sum = np.zeros((df_test.shape[0], 7))
    
    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape
    
    correlation_ensemble = []
    # Initialize vector containing predictions
    
    for i in range(1,11):
        
        tf.random.set_seed(i)
        
        # Define and compile model
        inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
        layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
        layer = Dropout(0.5)(layer)
        layer = BatchNormalization()(layer)
        layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
        layer = BatchNormalization()(layer)
        layer = Dropout(0.5)(layer)
        layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
        layer = Dropout(0.5)(layer)
        layer = Flatten()(layer)
        layer = Dense(300, activation='sigmoid')(layer)  # 300
        layer = Dropout(0.5)(layer)
        layer = Dense(200, activation='sigmoid')(layer)  # 300
        predictions = Dense(7, activation='linear')(layer)
        norm_predictions = CustomNormalization()(predictions)  # Assuming "predictions" is your existing output

        model = Model(inputs=inputs, outputs=norm_predictions)
        model.summary()

        # compile model
        model.compile(optimizer= Adam(clipvalue=1.0),  # You can adjust the clipvalue as needed
                    loss=MaskedMSE(mask_value=-1),
                    metrics=["mse"]
                    )

        # Run model
        history=model.fit(data_reader(input_path_train, batch_size=100),
                                epochs=20,
                                validation_data=data_reader(input_path_valid,batch_size=500),
                                callbacks=None,
                                verbose=1)    

        # We predict the test data
        predicted = model.predict(data_reader(input_path_test, batch_size=500))

        # Sum the predicted values (will be averaged at the end of the 10 iterations)
        predictions_sum =  np.add(predictions_sum, predicted)
        
        
        # We reed the data in the same order to compute the correlation score
        test_data = data_reader(input_path_test,batch_size=500)
        test_tensor = np.empty(shape=[0,7])
        for batch in test_data:
            test_tensor = np.append(test_tensor, batch[1], axis=0)
            
        # Append correlation coefficient and append to previous
        corr_coefficient = pearson_correlation_multi_head(test_tensor, predicted, mask_value=-1)
        correlation_ensemble.append(corr_coefficient)
        avg_corr_coefficient = 1
        
        
    # We fill the dataframe with predictions and fold annotation
    
    predicted = predictions_sum/10 # We divide by the number of iterations
    
    df_test["avg_prediction_State_1M"] = predicted[:,0]
    df_test["avg_prediction_State_2D"] = predicted[:,1]
    df_test["avg_prediction_State_3E"] = predicted[:,2]
    df_test["avg_prediction_State_4M"] = predicted[:,3]
    df_test["avg_prediction_State_5M"] = predicted[:,4]
    df_test["avg_prediction_State_6N"] = predicted[:,5]
    df_test["avg_prediction_State_7M"] = predicted[:,6]
    df_test["fold"] = str(i)
    df_test["partition"] = "test"
    corr_coefficients["fold"+str(i)] = avg_corr_coefficient

    
    # Append fold to previous folds
    df_test_10folds = pd.concat([df_test_10folds, df_test], ignore_index=True)    
    



# Save the results for all folds
df_test_10folds.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/ensemble_multihead_LibH_wide_pivot_state3_test_predicted_cv10fold.csv", index=False)

model_path = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/libH_test_multihead_last_cv.h5"
model.save_weights(model_path, save_format='h5') 

df_test_10folds

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 246, 4)]          0         
                                                                 
 conv1 (Conv1D)              (None, 240, 250)          7250      
                                                                 
 dropout (Dropout)           (None, 240, 250)          0         
                                                                 
 batch_normalization (Batch  (None, 240, 250)          1000      
 Normalization)                                                  
                                                                 
 conv2 (Conv1D)              (None, 233, 250)          500250    
                                                                 
 batch_normalization_1 (Bat  (None, 233, 250)          1000      
 chNormalization)                                            

2023-12-04 13:07:07.721338: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-12-04 13:07:07.740656: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-04 13:07:07.833822: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-04 13:07:10.083283: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f11d342c580 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-04 13:07:10.083323: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-12-04 13:07:10.087603: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-04 13:07:10.166928: I ./tensorflow/compiler/jit/device_comp

185/185 [==============================] - 11s 27ms/step - loss: 0.0558 - mse: 0.0898 - val_loss: 0.0506 - val_mse: 0.0830
Epoch 2/20
  7/185 [>.............................] - ETA: 3s - loss: 0.0366 - mse: 0.0697

2023-12-04 13:07:17.327471: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 18089997915302520933
2023-12-04 13:07:17.327535: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15502469122509296139
2023-12-04 13:07:17.327556: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 12370592359314852364


185/185 [==============================] - 4s 21ms/step - loss: 0.0332 - mse: 0.0674 - val_loss: 0.0477 - val_mse: 0.0774
Epoch 3/20
185/185 [==============================] - 4s 20ms/step - loss: 0.0324 - mse: 0.0665 - val_loss: 0.0459 - val_mse: 0.0776
Epoch 4/20
185/185 [==============================] - 4s 21ms/step - loss: 0.0311 - mse: 0.0653 - val_loss: 0.0307 - val_mse: 0.0638
Epoch 5/20
185/185 [==============================] - 4s 21ms/step - loss: 0.0302 - mse: 0.0645 - val_loss: 0.0298 - val_mse: 0.0638
Epoch 6/20
185/185 [==============================] - 4s 21ms/step - loss: 0.0299 - mse: 0.0641 - val_loss: 0.0247 - val_mse: 0.0574
Epoch 7/20
185/185 [==============================] - 4s 21ms/step - loss: 0.0293 - mse: 0.0635 - val_loss: 0.0244 - val_mse: 0.0585
Epoch 8/20
185/185 [==============================] - 4s 21ms/step - loss: 0.0291 - mse: 0.0635 - val_loss: 0.0314 - val_mse: 0.0667
Epoch 9/20
185/185 [==============================] - 4s 20ms/step - loss: 0.028

...

2023-12-04 15:19:04.367538: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4074747284500932603


Model: "model_98"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 246, 4)]          0         
                                                                 
 conv1 (Conv1D)              (None, 240, 250)          7250      
                                                                 
 dropout_490 (Dropout)       (None, 240, 250)          0         
                                                                 
 batch_normalization_392 (B  (None, 240, 250)          1000      
 atchNormalization)                                              
                                                                 
 conv2 (Conv1D)              (None, 233, 250)          500250    
                                                                 
 batch_normalization_393 (B  (None, 233, 250)          1000      
 atchNormalization)                                       

2023-12-04 15:20:31.087173: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 18148647446109167719


185/185 [==============================] - 7s 21ms/step - loss: 0.0498 - mse: 0.0834 - val_loss: 0.0464 - val_mse: 0.0817
Epoch 2/20
185/185 [==============================] - 4s 20ms/step - loss: 0.0335 - mse: 0.0672 - val_loss: 0.0463 - val_mse: 0.0821
Epoch 3/20
185/185 [==============================] - 4s 20ms/step - loss: 0.0318 - mse: 0.0656 - val_loss: 0.0461 - val_mse: 0.0820
Epoch 4/20
185/185 [==============================] - 4s 20ms/step - loss: 0.0310 - mse: 0.0648 - val_loss: 0.0319 - val_mse: 0.0690
Epoch 5/20
185/185 [==============================] - 4s 20ms/step - loss: 0.0305 - mse: 0.0646 - val_loss: 0.0299 - val_mse: 0.0681
Epoch 6/20
185/185 [==============================] - 4s 20ms/step - loss: 0.0298 - mse: 0.0638 - val_loss: 0.0310 - val_mse: 0.0694
Epoch 7/20
185/185 [==============================] - 4s 20ms/step - loss: 0.0296 - mse: 0.0634 - val_loss: 0.0285 - val_mse: 0.0673
Epoch 8/20
185/185 [==============================] - 4s 20ms/step - loss: 0.029

,State_3E,seq,avg_prediction_State_1M,avg_prediction_State_2D,avg_prediction_State_3E,avg_prediction_State_4M,avg_prediction_State_5M,avg_prediction_State_6N,avg_prediction_State_7M,Seq,CRS,State_1M,State_2D,State_4M,State_5M,State_6N,State_7M,fold,partition
0,0.167140,NaN,0.034122,0.056956,0.082613,0.014942,-0.017703,0.010972,0.087615,AGGACCGGATCAACTAAAAAATATTCCAATATCCCGAAGGAATGTT...,LibH.Seq17946,-0.022712,0.150933,0.016367,0.170019,0.062101,0.512236,10,test
1,0.571040,NaN,0.059414,0.064220,0.089638,0.024446,-0.004747,0.025203,0.079003,AGGACCGGATCAACTAAAACGGTTGGTATCGAGCAAAAGTTTCGAC...,LibH.Seq9908,0.315627,0.499681,0.332730,0.293306,0.357546,0.745269,10,test
2,-0.061613,NaN,0.037516,0.019479,0.030702,0.000215,-0.040058,-0.000698,0.026873,AGGACCGGATCAACTAAAAGGAATTCTACGGTGTGTGAGCAACGAC...,LibH.Seq10079,-0.040314,-0.002430,0.010253,-0.029671,0.103227,-0.024994,10,test
3,-0.135387,NaN,0.075212,0.057471,0.078330,0.017216,-0.011592,0.026352,0.058846,AGGACCGGATCAACTAAAAGGCTATCTCCTCTGTCGTTGACGGAGG...,LibH.Seq102,-0.060058,-0.123311,-0.054406,-0.249021,-0.094023,-0.097925,10,test
4,0.006998,NaN,0.258748,0.144842,0.188201,0.010700,-0.014790,0.032502,0.090744,AGGACCGGATCAACTAAAAGTTGTTATGCAATCGAATTGTCCCAGA...,LibH.Seq1563,0.369278,-0.059458,-0.080450,-0.144695,-0.076475,-0.130017,10,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20462,0.037999,NaN,0.103466,0.139073,0.186994,0.068268,0.027565,0.046775,0.176322,AGGACCGGATCAACTTTTTGAGACAGTGCATATATGGAATATGAGT...,LibH.Seq14658,-0.004687,0.127145,0.026934,0.003207,-0.039653,0.015313,10,test
20463,0.637832,NaN,0.228847,0.304547,0.405892,0.147898,0.114398,0.115509,0.391720,AGGACCGGATCAACTTTTTGATAATTAGCCTTTTGACCCGCTTAGT...,LibH.Seq17905,0.379286,0.544286,0.212274,0.318147,0.352252,0.808867,10,test
20464,-0.095661,NaN,0.038730,-0.003271,0.018227,-0.002462,-0.042528,0.008901,-0.000550,AGGACCGGATCAACTTTTTGGACGTCGGAAAGCACGTATAATATGA...,LibH.Seq11643,-0.122371,-0.090954,-0.086833,-0.115017,-0.029930,-0.134379,10,test
20465,-0.074385,NaN,0.023824,-0.009137,0.012744,-0.004441,-0.043020,0.013114,-0.005499,AGGACCGGATCAACTTTTTTAAGGAGCGTGCTTCATGTGCATACCG...,LibH.Seq12043,-0.102437,-0.105537,-0.024417,-0.046467,-0.022182,-0.048959,10,test


In [ ]:
corr_coefficients

In [ ]:
from tensorflow.keras.optimizers import Adam

corr_coefficients = {}

# We iterate through each of the train folds to train, test and validate the model
for i in range(1,11):
    #Define inputs
    input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_CV"+str(i)+"_LibA_wide_pivot_state3_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_CV"+str(i)+"_LibA_wide_pivot_state3_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_CV"+str(i)+"_LibA_wide_pivot_state3_test.csv"
    
    # Read test data to then predict
    df_test = pd.read_csv(input_path_test)
    df_test["fold"] = str(i)
    
    predictions_sum = np.zeros((df_test.shape[0], 7))
    
    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape
    
    # Define and compile model
    inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
    layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
    layer = Dropout(0.5)(layer)
    layer = BatchNormalization()(layer)
    layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
    layer = BatchNormalization()(layer)
    layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
    layer = Dropout(0.3)(layer)
    layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
    layer = BatchNormalization()(layer)
    layer = Dropout(0.5)(layer)
    layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
    layer = BatchNormalization()(layer)
    layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
    layer = Dropout(0.5)(layer)
    layer = Flatten()(layer)
    layer = Dense(300, activation='sigmoid')(layer)  # 300
    layer = Dropout(0.5)(layer)
    layer = Dense(200, activation='sigmoid')(layer)  # 300
    predictions = Dense(7, activation='linear')(layer)
    norm_predictions = CustomNormalization()(predictions)  # Assuming "predictions" is your existing output

    model = Model(inputs=inputs, outputs=norm_predictions)
    model.summary()

    # compile model
    model.compile(optimizer= Adam(clipvalue=1.0),  # You can adjust the clipvalue as needed
                loss=MaskedMSE(mask_value=-1),
                metrics=["mse"]
                )

    # Run model
    history=model.fit(data_reader(input_path_train, batch_size=100),
                            epochs=20,
                            validation_data=data_reader(input_path_valid,batch_size=500),
                            callbacks=None,
                            verbose=1)    

    # We predict the test data
    predicted = model.predict(data_reader(input_path_test, batch_size=500))
    
    # We reed the data in the same order to compute the correlation score
    test_data = data_reader(input_path_test,batch_size=500)
    test_tensor = np.empty(shape=[0,7])
    for batch in test_data:
        test_tensor = np.append(test_tensor, batch[1], axis=0)
        
    # Append correlation coefficient and append to previous
    corr_coefficient = pearson_correlation_multi_head(test_tensor, predicted, mask_value=-1)
    avg_corr_coefficient = 1
        
    df_test["avg_prediction_State_1M"] = predicted[:,0]
    df_test["avg_prediction_State_2D"] = predicted[:,1]
    df_test["avg_prediction_State_3E"] = predicted[:,2]
    df_test["avg_prediction_State_4M"] = predicted[:,3]
    df_test["avg_prediction_State_5M"] = predicted[:,4]
    df_test["avg_prediction_State_6N"] = predicted[:,5]
    df_test["avg_prediction_State_7M"] = predicted[:,6]
    df_test["fold"] = str(i)
    df_test["partition"] = "test"
    corr_coefficients["fold"+str(i)] = avg_corr_coefficient
    
    # Append fold to previous folds
    df_test_10folds = pd.concat([df_test_10folds, df_test], ignore_index=True)
    break

# Save the results for all folds
df_test_10folds.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/ensemble_multihead_LibA_wide_pivot_state3_test_predicted_cv10fold.csv", index=False)

model_path = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/test_multihead_last_cv.h5"
model.save_weights(model_path, save_format='h5') 

df_test_10folds

In [ ]:
df_test_10folds
